# Logic and Model-Based Reasoning

In [27]:
from utils import *
from logic import *
from copy  import *

**Notes on Logic**

Syntax:

Logical **not** (negation): $\neg$, ~

Logical **and** (conjunction): $\wedge$, &

Logical **or** (disjunction): $\vee$, |

De Morgan’s Theorem

$\neg(A \wedge B) => (\neg A) \vee (\neg B)$

$\neg(A \vee B) => (\neg A) \wedge (\neg B)$

...

# Practice reducing sentances to CNF
The cell below will reduce logical sentances to CNF.  This is useful to check practice problems.  The following syntax is used:

Implies: ==>, >>

IFF: <=>

and: &

not: ~

or: |

The flag SHOW_STEPS controls how much of the process is displayed. 

SHOW_STEPS = False - Output is only final answer

SHOW_STEPS = True - Output shows each step in the reduction process

In [28]:
#Examples
from logic import *
s = "A==>B"
to_cnf(s)

(B | ~A)

In [29]:
s = "A<=>B"
to_cnf(s)

((A | ~B) & (B | ~A))

In [30]:
s = "(A&B)|(C&D)"
to_cnf(s)

((C | A) & (D | A) & (C | B) & (D | B))

In [31]:
s = "(A<=>B)&(B==>C)"
to_cnf(s, True)

Initial Expression:
((A <=> B) & (B >> C))
Eliminate Implications:
(((A | ~B) & (B | ~A)) & (C | ~B))
Distribution
((A | ~B) & (B | ~A) & (C | ~B))


((A | ~B) & (B | ~A) & (C | ~B))

In [32]:
s = "(A<=>B)|~(D&C)"
to_cnf(s, True)

Initial Expression:
((A <=> B) | ~(D & C))
Eliminate Implications:
(((A | ~B) & (B | ~A)) | ~(D & C))
De Morgan's Theorem:
(((A | ~B) & (B | ~A)) | (~D | ~C))
Distribution
((A | ~B | ~D | ~C) & (B | ~A | ~D | ~C))


((A | ~B | ~D | ~C) & (B | ~A | ~D | ~C))

In [33]:
#HW 9 Problems

In [34]:
#B.1
SHOW_STEPS = True
s = "~((~A&B)|(C&D))"
to_cnf(s, SHOW_STEPS)

Initial Expression:
~((~A & B) | (C & D))
De Morgan's Theorem:
((A | ~B) & (~C | ~D))


((A | ~B) & (~C | ~D))

In [35]:
#B.2
s = "A<=>A"
to_cnf(s, SHOW_STEPS)

Initial Expression:
(A <=> A)
Eliminate Implications:
((A | ~A) & (A | ~A))


((A | ~A) & (A | ~A))

In [36]:
#B.3
s = "(A<=>B)|C"
to_cnf(s, SHOW_STEPS)

Initial Expression:
((A <=> B) | C)
Eliminate Implications:
(((A | ~B) & (B | ~A)) | C)
Distribution
((A | ~B | C) & (B | ~A | C))


((A | ~B | C) & (B | ~A | C))

In [37]:
SHOW_STEPS = True
s = "~((~A&B)|(C&D))"
to_cnf(s, SHOW_STEPS)

Initial Expression:
~((~A & B) | (C & D))
De Morgan's Theorem:
((A | ~B) & (~C | ~D))


((A | ~B) & (~C | ~D))

In [38]:
#Demonstrate the steps of the unit propogation

In [39]:
#Split a sentance into clauses
s = "~((~A&B)|(~C&D)|C|B)"
clauses = conjuncts(to_cnf(s))
to_cnf(s)

((A | ~B) & (C | ~D) & ~C & ~B)

In [40]:
#To find a valid model of the sentance, all clauses must be true
#Look for clauses with single values, and add their negations to the list of expressions that must be false
false_exp = []
true_exp = []
for c in clauses:
    if len(c.args)<=1:
        true_exp.append(c)
        false_exp.append(to_cnf(~c))
        print(false_exp)

[C]
[C, B]


In [41]:
#Remove already satisfied clases
for i in true_exp:
    clauses.remove(i)

print clauses

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(clauses)? (<ipython-input-41-35a09d047ac6>, line 5)

In [42]:
cp_clauses = deepcopy(clauses)
#Remove all literals known to be false from the other clases
for c in cp_clauses:
    for fe in false_exp:
        if fe in deepcopy(c.args):
            c.args.remove(fe)

print(cp_clauses)

#If any clauses are remaining with a single literal, add that literal to the known true/fault lists repeat
print(false_exp)
for c in cp_clauses:
    print(c)
    repeat = False
    if len(c.args) <= 1:
        if len(c.args) == 1:
            c =  expr(c.args[0].__repr__())
            print(c)
        true_exp.append(c)
        false_exp.append(to_cnf(~c))
        repeat = True

print(true_exp)
        


            

[(A | ~B), |~D, ~, ~]
[C, B]
(A | ~B)
|~D
~D
~


IndexError: list index out of range

In [48]:
def trim_or(clauses):
    #print type(clauses)
    #print "TRIMMING %s" % clauses

    clauses_to_trim = []
    for c in clauses:
        if len(c.args)==1 and (c.op == "|" or c.op == "|~"):
            clauses_to_trim.append(c)
    #print "ltt: %s"%len(clauses_to_trim)
    if len(clauses_to_trim) > 0:
        clauses = [c.args[0] if (c in clauses_to_trim) else c for c in clauses]
    #print "TRIMMED: %s"%clauses
    return clauses

def unit_propagation(s):
    #print type(s)
    #if type(s) == "<type 'str'>":
    s_cnf = to_cnf(s)
    clauses = conjuncts(s_cnf)
    #else:
    #    clauses = s
            
    repeat = True
    false_exp = []
    true_exp = []
    flag = True
    while (flag or repeat):
        #print "Loop Start"
        #print clauses
        if not repeat:
            flag = False
        #for c in clauses:
        #    if len(c.args)<=1:
        #        true_exp.append(c)
        #        false_exp.append(to_cnf(~c))
        repeat = False
        for c in clauses:
            #print len(clauses)
            #print "c: %s"%c
            if len(c.args) <= 1 and len(clauses) > 0:
                c2 = deepcopy(c)
                #print "c2:%s"%c2
                [c2] = trim_or([c2])
                #print "c2:%s"%c2
                if c not in true_exp:
                    #trim_or([c2])
                    true_exp.append(c2)
                    #print true_exp
                    false_exp.append(to_cnf(~c))
                    repeat = True 
                
        for e in true_exp:
            #print "e:%s"%e
            if e in clauses: clauses.remove(e) 
            #print "Clauses:%s"%clauses
            
        for c in clauses:
            for fe in false_exp:
                if fe in deepcopy(c.args):
                    #print ("Args:%s"%c.args)
                    #print ("Op:%s"%c.op)
                    if c.op != "~":
                        c.args.remove(fe)
                    #print "Clause after removing %s: %s"%(fe, clauses)
        clauses = trim_or(clauses)
    
    return clauses, true_exp

s = "(A1 ==> (X1 <=>(P1&V1)))&P1&~V1&A1"
print(unit_propagation(s))

([(P1 | ~X1)], [P1, ~V1, A1, ~X1])


In [50]:
s = "~((~A&B)|(~C&D)|~C|B)"
print(to_cnf(s))

((A | ~B) & (C | ~D) & C & ~B)


In [51]:
#Boolean Spaceship Example
#Model that encodes the relationship between parts
C = []
C.append("A1 ==> (X1 <=>(P1&V1))")
C.append("A2 ==> (X2 <=>(P2&V2))")
C.append("A3 ==> (T1 <=>(X1&X2))")
C.append("A4 ==> (X3 <=>(V3&P1))")
C.append("A5 ==> (X4 <=>(V4&P2))")
C.append("A6 ==> (T2 <=>(X3&X4))")
C.append("A7 ==> (S1 <=>((T1&Y3)|(T2&Y3)))")
#C.append("R1 ==> (Y3 <=>((Y1|Y2)&(~Y1|~Y2)))")
C.append("R1 ==> (Y3 <=>((Y1|Y2)))")
C.append("C1 ==> (Y1 <=>B1)")
C.append("C2 ==> (Y2 <=>B2)")


def make_sentance(clauses):
    s = ""
    for c in clauses:
        repeat = False
        if len(c.args) <= 1:
            if len(c.args) == 1:
                c =  expr(c.args[0].__repr__())
            true_exp.append(c)
            false_exp.append(to_cnf(~c))
            repeat = True 
        
    return clauses, true_exp

s = "(A1 ==> (X1 <=>(P1&V1)))&P1&V1&A1"
print(unit_propagation(s))

([], [P1, V1, A1, X1])


In [52]:
s = "~((~A&B)|(~C&D)|C|B)"
print(to_cnf(s))

((A | ~B) & (C | ~D) & ~C & ~B)


In [57]:
#Boolean Spaceship Example
#Model that encodes the relationship between parts
C = []
C.append("A1 ==> (X1 <=>(P1&V1))")
C.append("A2 ==> (X2 <=>(P2&V2))")
C.append("A3 ==> (T1 <=>(X1&X2))")
C.append("A4 ==> (X3 <=>(V3&P1))")
C.append("A5 ==> (X4 <=>(V4&P2))")
C.append("A6 ==> (T2 <=>(X3&X4))")
C.append("A7 ==> (S1 <=>((T1|T2)&Y3))")
#C.append("R1 ==> (Y3 <=>((Y1|Y2)&(~Y1|~Y2)))")
C.append("R1 ==> (Y3 <=>((Y1|Y2)))")
C.append("C1 ==> (Y1 <=>B1)")
C.append("C2 ==> (Y2 <=>B2)")


def make_sentance(clauses):
        s = ""
        for c in clauses:
            if s == "":
                s = "("+c+")"
            else:        
                s += "&("+c+")"
        return s

print(to_cnf(make_sentance(C)))

#All components functioning
C.append("A1")
C.append("A2")
C.append("A3")
C.append("A4")
C.append("A5")
C.append("A6")
C.append("A7")
C.append("R1")
C.append("C1")
C.append("C2")


print(unit_propagation(make_sentance(C)))


((X1 | ~P1 | ~V1 | ~A1) & (P1 | ~X1 | ~A1) & (V1 | ~X1 | ~A1) & (X2 | ~P2 | ~V2 | ~A2) & (P2 | ~X2 | ~A2) & (V2 | ~X2 | ~A2) & (T1 | ~X1 | ~X2 | ~A3) & (X1 | ~T1 | ~A3) & (X2 | ~T1 | ~A3) & (X3 | ~V3 | ~P1 | ~A4) & (V3 | ~X3 | ~A4) & (P1 | ~X3 | ~A4) & (X4 | ~V4 | ~P2 | ~A5) & (V4 | ~X4 | ~A5) & (P2 | ~X4 | ~A5) & (T2 | ~X3 | ~X4 | ~A6) & (X3 | ~T2 | ~A6) & (X4 | ~T2 | ~A6) & (S1 | (~T1 & ~T2) | ~Y3 | ~A7) & (T1 | T2 | ~S1 | ~A7) & (Y3 | ~S1 | ~A7) & (~Y1 | Y3 | ~R1) & (~Y2 | Y3 | ~R1) & (Y1 | Y2 | ~Y3 | ~R1) & (Y1 | ~B1 | ~C1) & (B1 | ~Y1 | ~C1) & (Y2 | ~B2 | ~C2) & (B2 | ~Y2 | ~C2))
([(X1 | ~P1 | ~V1), (P1 | ~X1), (V1 | ~X1), (X2 | ~P2 | ~V2), (P2 | ~X2), (V2 | ~X2), (T1 | ~X1 | ~X2), (X1 | ~T1), (X2 | ~T1), (X3 | ~V3 | ~P1), (V3 | ~X3), (P1 | ~X3), (X4 | ~V4 | ~P2), (V4 | ~X4), (P2 | ~X4), (T2 | ~X3 | ~X4), (X3 | ~T2), (X4 | ~T2), (S1 | (~T1 & ~T2) | ~Y3), (T1 | T2 | ~S1), (Y3 | ~S1), (~Y1 | Y3), (~Y2 | Y3), (Y1 | Y2 | ~Y3), (Y1 | ~B1), (B1 | ~Y1), (Y2 | ~B2), (B2 | ~Y2)], [A1, A2, 

In [58]:
#Assign a state to the system
#Propellant on
C.append("P1")
C.append("P2")

#Battery 1 on
C.append("B1")

#Battery 2 off
C.append("~B2")

#Valve 1 and 2 open
C.append("V1")
C.append("V2")

#Valve 2 and 3 closed
C.append("~V3")
C.append("~V4")

#print to_cnf(make_sentance(C))

unit_propagation(make_sentance(C))


([(P1 | ~X3),
  (P2 | ~X4),
  (~X3 | ~X4),
  (S1 | (~T1 & ~T2)),
  (T1 | ~S1),
  (Y3 | ~S1),
  (~Y2 | Y3)],
 [A1,
  A2,
  A3,
  A4,
  A5,
  A6,
  A7,
  R1,
  C1,
  C2,
  P1,
  P2,
  B1,
  ~B2,
  V1,
  V2,
  ~V3,
  ~V4,
  X1,
  X2,
  ~X3,
  ~X4,
  Y1,
  ~Y2,
  T1,
  ~T2,
  Y3])